# Electricistan

In [2]:
import pandas as pd

In [4]:
orig_data = pd.read_csv("./data/train.csv")

In [5]:
print(orig_data.head(4))

              datetime  power
0  2012.01.01 00:15:00   3767
1  2012.01.01 00:30:00   3743
2  2012.01.01 00:45:00   3691
3  2012.01.01 01:00:00   3645
